<a href="https://colab.research.google.com/github/Brian0615/CSC413FinalProject/blob/brian/HMSA_Inference/HMSA_Reproduce_Results_Cityscapes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HMSA - Reproducing Evaluation Results on the Cityscapes Dataset
## Overview
This file is used to reproducing results from the Hierarchical Multi-Scale Attention paper. Essentially, this file runs the evaluation script provided by the paper's authors as a simple test to confirm the results that the authors reported, and also whether we are able to use the code in Colab.

## Before Running the Code
1. Ensure that the runtime is set to GPU!
2. Ensure that the Colab VM is in the same region as the Google Cloud Storage Buckets (which are `US-multi-region`).
  - This minimizes costs, as Google charges for network egress.
  - The Colab region can be checked by running the cell below.
  - Perform a factory reset of the runtime until the runtime is in the US.

In [1]:
!curl ipinfo.io

{
  "ip": "35.199.6.240",
  "hostname": "240.6.199.35.bc.googleusercontent.com",
  "city": "Washington",
  "region": "Washington, D.C.",
  "country": "US",
  "loc": "38.8951,-77.0364",
  "org": "AS15169 Google LLC",
  "postal": "20045",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

## 1. Setup


### 1.1. Clone Git Repo

In [2]:
!git clone https://github.com/Brian0615/CSC413FinalProject.git
%cd CSC413FinalProject/
!git checkout brian

Cloning into 'CSC413FinalProject'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (408/408), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 408 (delta 91), reused 344 (delta 62), pack-reused 0
Receiving objects: 100% (408/408), 21.99 MiB | 40.87 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/CSC413FinalProject
Branch 'brian' set up to track remote branch 'brian' from 'origin'.
Switched to a new branch 'brian'


###1.2. Download Cityscapes Dataset

In [28]:
%cd /content
!mkdir cityscapes
%cd cityscapes
!mkdir leftImg8bit_trainvaltest
!mkdir gtFine_trainvaltest
%cd leftImg8bit_trainvaltest
!mkdir leftImg8bit
%cd leftImg8bit
!mkdir train
!mkdir val
!mkdir test

/content
mkdir: cannot create directory ‘cityscapes’: File exists
/content/cityscapes
mkdir: cannot create directory ‘leftImg8bit_trainvaltest’: File exists
/content/cityscapes/leftImg8bit_trainvaltest
mkdir: cannot create directory ‘leftImg8bit’: File exists
/content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit
mkdir: cannot create directory ‘train’: File exists
mkdir: cannot create directory ‘val’: File exists
mkdir: cannot create directory ‘test’: File exists


In [5]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapi

In [29]:
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/gtFine /content/cityscapes/gtFine_trainvaltest/

Streaming output truncated to the last 5000 lines.
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051574_gtFine_labelIds.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051574_gtFine_polygons.json...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_color.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_instanceIds.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_labelIds.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051661_gtFine_polygons.json...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_051877_gtFine_color.png...
Copying gs://csc413-final-project-cityscapes-data/gtFine/train/strasbourg/strasbourg_000001_0518

In [10]:
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/leftImg8bit/train/ /content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/leftImg8bit/val/ /content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit/val/
!gsutil -m cp -r gs://csc413-final-project-cityscapes-data/leftImg8bit/test/ /content/cityscapes/leftImg8bit_trainvaltest/leftImg8bit/test/

Streaming output truncated to the last 5000 lines.
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000008_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000003_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000004_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000009_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000010_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000011_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000012_000019_leftImg8bit.png...
Copying gs://csc413-final-project-cityscapes-data/leftImg8bit/train/aachen/aachen_000013_000019_leftImg8bit.png...
Copying gs://csc413-final-pro

### 1.3. Download Weights


In [18]:
!mkdir /content/seg_weights/
!gsutil -m cp gs://csc413-final-project-hmsa-weights/original_weights/cityscapes_ocrnet.HRNet_Mscale_outstanding-turtle.pth /content/seg_weights/cityscapes_ocrnet.HRNet_Mscale_outstanding-turtle.pth
!gsutil -m cp gs://csc413-final-project-hmsa-weights/original_weights/hrnetv2_w48_imagenet_pretrained.pth /content/seg_weights/hrnetv2_w48_imagenet_pretrained.pth

Copying gs://csc413-final-project-hmsa-weights/original_weights/cityscapes_ocrnet.HRNet_Mscale_outstanding-turtle.pth...
| [1/1 files][551.3 MiB/551.3 MiB] 100% Done                                    
Operation completed over 1 objects/551.3 MiB.                                    
Copying gs://csc413-final-project-hmsa-weights/original_weights/hrnetv2_w48_imagenet_pretrained.pth...
\ [1/1 files][296.2 MiB/296.2 MiB] 100% Done                                    
Operation completed over 1 objects/296.2 MiB.                                    


### 1.4. Install Dependencies

In [11]:
# install basic dependencies
!pip install runx==0.0.6 numpy sklearn h5py jupyter scikit-image pillow piexif cffi tqdm dominate opencv-python nose ninja
!apt-get update
!apt-get install libgtk2.0-dev -y
!rm -rf /var/lib/apt/lists/*

     |████████████████████████████████| 163kB 27.3MB/s 
     |████████████████████████████████| 112kB 53.9MB/s 
     |████████████████████████████████| 122kB 46.5MB/s 
     |████████████████████████████████| 645kB 47.0MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/r

In [12]:
# change cuda version to 10.1
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda  # replace symlink to cuda-11.0 with cuda-10.1

/usr/local


In [13]:
# install apex
%cd /home/
!git clone https://github.com/NVIDIA/apex.git apex
%cd apex
!python setup.py install --cuda_ext --cpp_ext

/home
Cloning into 'apex'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 7939 (delta 5), reused 12 (delta 0), pack-reused 7913
Receiving objects: 100% (7939/7939), 14.02 MiB | 26.89 MiB/s, done.
Resolving deltas: 100% (5404/5404), done.
/home/apex


torch.__version__  = 1.8.1+cu101


setup.py:67: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-inf

### 1.5. Config Setup
 - Inside `CSC413FinalProject/HMSA/config.py`, set the following items:
  ```
  __C.ASSETS_PATH = '/content'
  __C.DATASET.CITYSCAPES_DIR = os.path.join(__C.ASSETS_PATH, 'cityscapes')
  ```

In [14]:
# check GPU specs
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a7918aa7-b246-f4be-a13d-b24ca5e4284e)


* Since Colab only has one GPU, we need to adjust the evaluation script
* Inside `CSC413FinalProject/HMSA/scripts/eval_cityscapes.yml`, set the following:
  ```
  CMD: "python -m torch.distributed.launch --nproc_per_node=1 train.py"
  ```
  (i.e. change `--nproc_per_node=8` to `--nproc_per_node=1`)

## 2. Inference on Cityscapes

In [15]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=87fe2b925cf06fbca262e51814c8794eaee6d927cb3a660926c7b15b9f0cec2c
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  |     Proc size: 121.3 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total     15109MB


In [31]:
%cd /content/CSC413FinalProject/HMSA/

/content/CSC413FinalProject/HMSA


In [32]:
# dry run (to see full command)
!python -m runx.runx scripts/eval_cityscapes.yml -i -n

python -m torch.distributed.launch --nproc_per_node=1 train.py --dataset cityscapes --cv 0 --syncbn --apex --fp16 --bs_val 2 --eval val --n_scales 0.5,1.0,2.0 --snapshot ASSETS_PATH/seg_weights/cityscapes_ocrnet.HRNet_Mscale_outstanding-turtle.pth --arch ocrnet.HRNet_Mscale --result_dir logs/eval_cityscapes/remarkable-frigatebird_2021.04.11_16.04 


In [38]:
# real run
!python -m runx.runx scripts/eval_cityscapes.yml -i

None
Global Rank: 0 Local Rank: 0
Torch version: 1.8, 1.8.1+cu101
n scales [1.0]
dataset = cityscapes
ignore_label = 255
num_classes = 19
cv split val 0 ['val/lindau', 'val/frankfurt', 'val/munster']
mode val found 500 images
cn num_classes 19
Using Cross Entropy Loss
Using Cross Entropy Loss
Loading weights from: checkpoint=/content/seg_weights/cityscapes_ocrnet.HRNet_Mscale_outstanding-turtle.pth
=> init weights from normal distribution
=> loading pretrained model /content/seg_weights/hrnetv2_w48_imagenet_pretrained.pth
Trunk: hrnetv2
Model params = 72.1M
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processi

In [39]:
# upload results to google cloud bucket
!gsutil -m cp -r /content/CSC413FinalProject/HMSA/logs/eval_cityscapes/* gs://csc413-final-project-cityscapes-data/inference_results/

Copying file:///content/CSC413FinalProject/HMSA/logs/eval_cityscapes/fluffy-ibex_2021.04.11_17.32/best_checkpoint_ep0.pth [Content-Type=application/octet-stream]...
Copying file:///content/CSC413FinalProject/HMSA/logs/eval_cityscapes/fluffy-ibex_2021.04.11_17.32/hparams.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file: